# Unetのセグメンテーションモデル

In [ ]:
pip install segmentation_models_pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 813.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.4 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=ef8fd262620fabc2db9b6173c8e137577661b73005e3ced1ff4cafdfb4c69e90
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=56d6bf54ffcf8f6f5213cf920ea85a574ad4d4b1eb20858f00f1914eeced4dac
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytor

In [ ]:
pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.5 MB/s eta 0:00:00


In [ ]:
pip install timm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import numpy as np
import os
import glob
from tqdm import tqdm
import gc

import torchvision
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from fastai.vision.all import *
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
CV = 5
SEED = 777
fold = 1
PATCH_SIZE = 512
patch_size = 64
TH = .5
SEG_TRAIN = False
SEG = {
    'BS':16,
    'LR':5e-4,
    'EPOCHS':10
}
INF = {
    'BS':64,
    'LR':1e-5,
    'EPOCHS':10
}

In [ ]:
!unzip /content/drive/MyDrive/rsna-2024-lumbar-spine-degenerative-classification.zip -d /content #RSNA2024を今のディレクトリに展開する

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: /content/train_images/885894528/2509467535/43.dcm  
  inflating: /content/train_images/885894528/2509467535/44.dcm  
  inflating: /content/train_images/885894528/2509467535/45.dcm  
  inflating: /content/train_images/885894528/2509467535/46.dcm  
  inflating: /content/train_images/885894528/2509467535/47.dcm  
  inflating: /content/train_images/885894528/2509467535/48.dcm  
  inflating: /content/train_images/885894528/2509467535/49.dcm  
  inflating: /content/train_images/885894528/2509467535/5.dcm  
  inflating: /content/train_images/885894528/2509467535/50.dcm  
  inflating: /content/train_images/885894528/2509467535/6.dcm  
  inflating: /content/train_images/885894528/2509467535/7.dcm  
  inflating: /content/train_images/885894528/2509467535/8.dcm  
  inflating: /content/train_images/885894528/2509467535/9.dcm  
  inflating: /content/train_images/885894528/3303042519/1.dcm  
  inflating: /content/train_images/885894528/3303042519/10.dcm  

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class myUNet(nn.Module):
    def __init__(self):
        super(myUNet, self).__init__()

        self.UNet = smp.Unet(
            encoder_name="resnet18",
            classes=5,
            in_channels=1
        ).to(device)

    def forward(self,X):
        x = self.UNet(X)
#       MinMaxScaling along the class plane to generate a heatmap
        min_values = x.view(-1,5,PATCH_SIZE*PATCH_SIZE).min(-1)[0].view(-1,5,1,1) # Bug, I've been MinMaxScaling with the wrong values
        max_values = x.view(-1,5,PATCH_SIZE*PATCH_SIZE).max(-1)[0].view(-1,5,1,1)
        x = (x - min_values)/(max_values - min_values)

        return x

In [ ]:
idx_map = torch.stack([torch.arange(PATCH_SIZE)]*PATCH_SIZE).to(device)
idx_map = torch.stack([idx_map,idx_map.T]).view(1,1,2,PATCH_SIZE,PATCH_SIZE)
class myLoss(nn.Module):
    def __init__(
            self,
            alpha=.5
        ):
        super().__init__()
        self.alpha = alpha

    def clone(self):
        return myLoss(self.alpha)
    def forward(
            self,
            y,# Predictions
            t # Targets
        ):
        mask_pred = y
        _,mask_true = t
#       理想的な分布を中心（骨に問題があると言われている座標）から1変数の正規分布を３次元空間上で回転させたような分布であると仮定する。
        s2 = s2 = torch.as_tensor([PATCH_SIZE/8]*5)
#       正規分布のパラメタを以下のように設定する
        A = -1/(2*s2).to(device)
        K = 1/torch.sqrt(2*math.pi*s2).to(device)
        mask_pred = mask_pred*K.view(1,5,1,1)
        mask = idx_map - mask_true.view(-1,5,2,1,1)
        mask = torch.exp((A.view(-1,5,1,1,1)*mask*mask).sum(2))*K.view(-1,5,1,1)
#       ロスは理想的な分布とのcosine類似度にする
        D = 1 - ((mask*mask_pred).sum())**2/((mask*mask).sum()*(mask_pred*mask_pred).sum())

        return D

In [ ]:
train = pd.read_csv('/content/train.csv')
train.tail()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild
1974,4290709089,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
diagnosis = list(filter(lambda x: x.find('foraminal') > -1, train.columns))
train = train[train[diagnosis].isnull().values.sum(1)==0].reset_index(drop=True)
train.tail()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1962,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1963,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1964,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1965,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild
1966,4290709089,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
train2=train[['study_id']+diagnosis]

In [ ]:
labels = {
    'Normal/Mild':0,
    'Moderate':1,
    'Severe':2
}

In [ ]:
df_meta_f = pd.read_csv('/content/train_series_descriptions.csv')
df_meta_f.tail()

,study_id,series_id,series_description
6289,4287160193,1507070277,Sagittal T2/STIR
6290,4287160193,1820446240,Axial T2
6291,4290709089,3274612423,Sagittal T2/STIR
6292,4290709089,3390218084,Axial T2
6293,4290709089,4237840455,Sagittal T1


In [ ]:
df_coor = pd.read_csv('/content/train_label_coordinates.csv')
df_coor.tail()

,study_id,series_id,instance_number,condition,level,x,y
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333
48691,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L5/S1,211.813953,259.534884


In [ ]:
LF=df_coor[df_coor['condition']=='Left Neural Foraminal Narrowing'][[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'x',
    'y'
]].sort_values([
    'study_id',
    'series_id',
    'level'
])[[
    'study_id',
    'series_id',
    'level',
    'instance_number',
    'x',
    'y'
]]
LF.tail()

,study_id,series_id,level,instance_number,x,y
48687,4290709089,4237840455,L1/L2,11,219.465940,97.831063
48688,4290709089,4237840455,L2/L3,12,205.340599,140.207084
48689,4290709089,4237840455,L3/L4,12,202.724796,181.013624
48690,4290709089,4237840455,L4/L5,12,202.933333,219.733333
48691,4290709089,4237840455,L5/S1,12,211.813953,259.534884


In [ ]:
LF = LF[[
    'study_id',
    'series_id',
    'instance_number',
    'x',
    'y'
]]

In [ ]:
LF[[
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]] = np.tile(LF[['x','y']].values.reshape(-1,1,5,2),(1,5,1,1)).reshape(-1,10)
LF = LF.drop(columns=['x','y']).drop_duplicates().reset_index(drop=True)
LF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1
4084,4284048608,1875151370,16,223.753846,142.030769,208.000000,193.230769,204.061538,252.307692,199.138462,298.584615,209.969231,344.861538
4085,4284048608,1875151370,17,223.753846,142.030769,208.000000,193.230769,204.061538,252.307692,199.138462,298.584615,209.969231,344.861538
4086,4287160193,327893304,4,189.077757,113.105882,178.795404,148.823529,175.548346,185.082353,181.501287,226.211765,195.571875,265.176471
4087,4290709089,4237840455,11,219.465940,97.831063,205.340599,140.207084,202.724796,181.013624,202.933333,219.733333,211.813953,259.534884
4088,4290709089,4237840455,12,219.465940,97.831063,205.340599,140.207084,202.724796,181.013624,202.933333,219.733333,211.813953,259.534884


In [ ]:
diagnosis = list(filter(lambda x: x.find('left_neural') > -1, train.columns))

In [ ]:
LF = LF.merge(train[['study_id']+diagnosis], left_on='study_id', right_on='study_id')
LF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,left_neural_foraminal_narrowing_l5_s1
4071,4284048608,1875151370,16,223.753846,142.030769,208.000000,193.230769,204.061538,252.307692,199.138462,298.584615,209.969231,344.861538,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe
4072,4284048608,1875151370,17,223.753846,142.030769,208.000000,193.230769,204.061538,252.307692,199.138462,298.584615,209.969231,344.861538,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe
4073,4287160193,327893304,4,189.077757,113.105882,178.795404,148.823529,175.548346,185.082353,181.501287,226.211765,195.571875,265.176471,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
4074,4290709089,4237840455,11,219.465940,97.831063,205.340599,140.207084,202.724796,181.013624,202.933333,219.733333,211.813953,259.534884,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4075,4290709089,4237840455,12,219.465940,97.831063,205.340599,140.207084,202.724796,181.013624,202.933333,219.733333,211.813953,259.534884,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
diagnosis = {x:x[5:] for x in diagnosis}
diagnosis

{'left_neural_foraminal_narrowing_l1_l2': 'neural_foraminal_narrowing_l1_l2',
 'left_neural_foraminal_narrowing_l2_l3': 'neural_foraminal_narrowing_l2_l3',
 'left_neural_foraminal_narrowing_l3_l4': 'neural_foraminal_narrowing_l3_l4',
 'left_neural_foraminal_narrowing_l4_l5': 'neural_foraminal_narrowing_l4_l5',
 'left_neural_foraminal_narrowing_l5_s1': 'neural_foraminal_narrowing_l5_s1'}

In [ ]:
LF = LF.rename(columns=diagnosis)
LF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1,neural_foraminal_narrowing_l1_l2,neural_foraminal_narrowing_l2_l3,neural_foraminal_narrowing_l3_l4,neural_foraminal_narrowing_l4_l5,neural_foraminal_narrowing_l5_s1
4071,4284048608,1875151370,16,223.753846,142.030769,208.000000,193.230769,204.061538,252.307692,199.138462,298.584615,209.969231,344.861538,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe
4072,4284048608,1875151370,17,223.753846,142.030769,208.000000,193.230769,204.061538,252.307692,199.138462,298.584615,209.969231,344.861538,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe
4073,4287160193,327893304,4,189.077757,113.105882,178.795404,148.823529,175.548346,185.082353,181.501287,226.211765,195.571875,265.176471,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
4074,4290709089,4237840455,11,219.465940,97.831063,205.340599,140.207084,202.724796,181.013624,202.933333,219.733333,211.813953,259.534884,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4075,4290709089,4237840455,12,219.465940,97.831063,205.340599,140.207084,202.724796,181.013624,202.933333,219.733333,211.813953,259.534884,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
RF = df_coor[df_coor['condition']=='Right Neural Foraminal Narrowing'][[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'x',
    'y'
]].sort_values([
    'study_id',
    'series_id',
    'level'
])[[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'x',
    'y'
]].drop_duplicates()
RF.tail()

,study_id,series_id,instance_number,level,x,y
48686,4290709089,4237840455,5,L1/L2,219.405706,95.459321
48682,4290709089,4237840455,4,L2/L3,208.106799,140.203404
48683,4290709089,4237840455,4,L3/L4,204.195639,182.791594
48684,4290709089,4237840455,4,L4/L5,208.381925,222.922987
48685,4290709089,4237840455,4,L5/S1,215.268102,261.657732


In [ ]:
centers = {}
for i in range(len(RF)):
    row = RF.iloc[i]
    centers[row['study_id']]={}
for i in range(len(RF)):
    row = RF.iloc[i]
    centers[row['study_id']][row['series_id']]={'L1/L2':[],'L2/L3':[],'L3/L4':[],'L4/L5':[],'L5/S1':[]}
for i in range(len(RF)):
    row = RF.iloc[i]
    centers[row['study_id']][row['series_id']][row['level']].append([row['x'],row['y']])

In [ ]:
coordinates = np.zeros((len(RF),10))
coordinates[:] = np.nan
for i in range(len(RF)):
    row = RF.iloc[i]
    for level in centers[row['study_id']][row['series_id']]:
        if len(centers[row['study_id']][row['series_id']][level]) > 0:
            center = np.array(centers[row['study_id']][row['series_id']][level]).mean(0)
            coordinates[
                i,
                {'L1/L2':0, 'L2/L3':2, 'L3/L4':4, 'L4/L5':6, 'L5/S1':8}[level]:{'L1/L2':0, 'L2/L3':2, 'L3/L4':4, 'L4/L5':6, 'L5/S1':8}[level]+2
            ] = center

In [ ]:
RF = RF[[
    'study_id',
    'series_id',
    'instance_number',
    'x',
    'y'
]]
RF[[
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]] = coordinates
RF = RF.drop(columns=['x','y']).drop_duplicates().reset_index(drop=True)
RF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1
4206,4287160193,327893304,10,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260
4207,4287160193,327893304,11,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260
4208,4287160193,327893304,12,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260
4209,4290709089,4237840455,5,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732
4210,4290709089,4237840455,4,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732


In [ ]:
RF = RF[RF[[
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]].isnull().values.sum(1)==0].reset_index(drop=True)
RF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1
4204,4287160193,327893304,10,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260
4205,4287160193,327893304,11,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260
4206,4287160193,327893304,12,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260
4207,4290709089,4237840455,5,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732
4208,4290709089,4237840455,4,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732


In [ ]:
diagnosis = list(filter(lambda x: x.find('right_neural_foraminal') > -1, train.columns))
RF = RF.merge(train[['study_id']+diagnosis], left_on='study_id', right_on='study_id')
RF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1,right_neural_foraminal_narrowing_l1_l2,right_neural_foraminal_narrowing_l2_l3,right_neural_foraminal_narrowing_l3_l4,right_neural_foraminal_narrowing_l4_l5,right_neural_foraminal_narrowing_l5_s1
4189,4287160193,327893304,10,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4190,4287160193,327893304,11,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4191,4287160193,327893304,12,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4192,4290709089,4237840455,5,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4193,4290709089,4237840455,4,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
diagnosis = {x:x[6:] for x in diagnosis}
diagnosis

{'right_neural_foraminal_narrowing_l1_l2': 'neural_foraminal_narrowing_l1_l2',
 'right_neural_foraminal_narrowing_l2_l3': 'neural_foraminal_narrowing_l2_l3',
 'right_neural_foraminal_narrowing_l3_l4': 'neural_foraminal_narrowing_l3_l4',
 'right_neural_foraminal_narrowing_l4_l5': 'neural_foraminal_narrowing_l4_l5',
 'right_neural_foraminal_narrowing_l5_s1': 'neural_foraminal_narrowing_l5_s1'}

In [ ]:
RF = RF.rename(columns=diagnosis)
RF.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1,neural_foraminal_narrowing_l1_l2,neural_foraminal_narrowing_l2_l3,neural_foraminal_narrowing_l3_l4,neural_foraminal_narrowing_l4_l5,neural_foraminal_narrowing_l5_s1
4189,4287160193,327893304,10,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4190,4287160193,327893304,11,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4191,4287160193,327893304,12,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4192,4290709089,4237840455,5,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4193,4290709089,4237840455,4,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
F = pd.concat([LF,RF],axis=0,ignore_index=True)
F.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1,neural_foraminal_narrowing_l1_l2,neural_foraminal_narrowing_l2_l3,neural_foraminal_narrowing_l3_l4,neural_foraminal_narrowing_l4_l5,neural_foraminal_narrowing_l5_s1
8265,4287160193,327893304,10,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
8266,4287160193,327893304,11,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
8267,4287160193,327893304,12,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
8268,4290709089,4237840455,5,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
8269,4290709089,4237840455,4,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [ ]:
F = F.merge(df_meta_f[['series_id','series_description']], left_on='series_id', right_on='series_id')
F.tail()

,study_id,series_id,instance_number,x_L1L2,y_L1L2,x_L2L3,y_L2L3,x_L3L4,y_L3L4,x_L4L5,y_L4L5,x_L5S1,y_L5S1,neural_foraminal_narrowing_l1_l2,neural_foraminal_narrowing_l2_l3,neural_foraminal_narrowing_l3_l4,neural_foraminal_narrowing_l4_l5,neural_foraminal_narrowing_l5_s1,series_description
8265,4287160193,327893304,10,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T1
8266,4287160193,327893304,11,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T1
8267,4287160193,327893304,12,190.991909,110.351387,177.866413,149.241744,176.894155,189.590489,182.727708,224.591810,199.256110,260.079260,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T1
8268,4290709089,4237840455,5,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T1
8269,4290709089,4237840455,4,219.405706,95.459321,208.106799,140.203404,204.195639,182.791594,208.381925,222.922987,215.268102,261.657732,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T1


In [ ]:
target = F.columns[-6:-1]
target

Index(['neural_foraminal_narrowing_l1_l2', 'neural_foraminal_narrowing_l2_l3',
       'neural_foraminal_narrowing_l3_l4', 'neural_foraminal_narrowing_l4_l5',
       'neural_foraminal_narrowing_l5_s1'],
      dtype='object')

In [ ]:
labels = {
    'Normal/Mild':0,
    'Moderate':1,
    'Severe':2
}

coor = [
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]

In [ ]:
def augment_image_and_centers(image,centers,alpha):
    '''
    # Randomly flip the image horizontally.
    if random.random() > .5:
      if random.random() > 1 - alpha:
        image = image.flip(-1)
        centers[:,0] = PATCH_SIZE - centers[:,0]
    # Randomly flip the image vertically.
    if random.random() > 0.5:
      if random.random() > 1 - alpha:
        image = image.flip(-2)
        centers[:,1] = PATCH_SIZE - centers[:,1]

    if random.random() > 1 - alpha:
      if random.random() > .5:
    #   Randomly flip the image
    #   Wich axis?
         axis = np.random.randint(2)
        image = image.flip(axis+1)
        centers[:,-1-axis] = PATCH_SIZE - centers[:,-1-axis]
    '''
#   Randomly rotate the image.
    angle = torch.as_tensor(random.uniform(-180, 180)*alpha)
    image = torchvision.transforms.functional.rotate(image,angle.item())
#   https://discuss.pytorch.org/t/rotation-matrix/128260
    angle = angle*math.pi/180
    s = torch.sin(angle)
    c = torch.cos(angle)
    rot = torch.stack([
        torch.stack([c, s]),
        torch.stack([-s, c])
      ])
    centers = ((centers.cpu() - PATCH_SIZE//2) @ rot) + PATCH_SIZE//2
    return image,centers

In [ ]:
class T1Dataset(Dataset):
    def __init__(self, df, VALID=False, alpha=0):
        self.data = df
        self.VALID = VALID
        self.alpha = alpha

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]

        centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()

        sample = '/content/train_images/'
        sample = sample+str(row['study_id'])+'/'+str(row['series_id'])+'/'+str(row['instance_number'])+'.dcm'
        image = pydicom.dcmread(sample).pixel_array
        H,W = image.shape
#       By plane resizing I've been distorting the proportions
        if H > W:
            d = W
            if not self.VALID:
                h = int((H - d)*(.5 + self.alpha*(.5 - np.random.rand())))
            else:
                h = (H - d)//2
            image = image[h:h+d]
            centers[:,1] -= h
            H = W
        elif H < W:
            d = H
            if not self.VALID:
                w = int((W - d)*(.5 + self.alpha*(.5 - np.random.rand())))
            else:
                w = (W - d)//2
            image = image[:,w:w+d]
            centers[:,0] -= w
            W = H
        image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
        image = torch.as_tensor(image/np.max(image)).unsqueeze(0).float()

        label = torch.as_tensor([labels[x] for x in row[target]])

        centers[:,0] = centers[:,0]*PATCH_SIZE/W
        centers[:,1] = centers[:,1]*PATCH_SIZE/H

        if not self.VALID: image,centers = augment_image_and_centers(image,centers,self.alpha)
        return image.to(device),[label.to(device),centers.to(device)]

In [ ]:
SEG_TRAIN = True

In [ ]:
import pickle

tdf = F[F['series_description'] != fold]
vdf = F[F['series_description'] == fold]

# tdf2=df_melted[df_melted['series_description'] != fold]
# vdf2=df_melted[df_melted['series_description'] == fold]

tds = T1Dataset(tdf)
vds = T1Dataset(vdf,VALID=True)
tdl = torch.utils.data.DataLoader(tds, batch_size=SEG['BS'], shuffle=True, drop_last=True)
vdl = torch.utils.data.DataLoader(vds, batch_size=SEG['BS'], shuffle=False)

if SEG_TRAIN:
    seed_everything(SEED)

    dls = DataLoaders(tdl,vdl)

    n_iter = len(tds)//SEG['BS']

    model = myUNet()
    learn = Learner(
        dls,
        model,
        lr=SEG['LR'],
        loss_func=myLoss(alpha=0.5),
        # cbs=[
        #     ShowGraphCallback(),
        #     alpha_cb
        # ]
    )
    learn.fit_one_cycle(SEG['EPOCHS'])
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+str(fold)+"_sagt1_rev"+".pkl", 'wb') as f:
      pickle.dump(model, f)
    del tdl,vdl,dls,model,learn
    gc.collect()

epoch,train_loss,valid_loss,time


KeyboardInterrupt: 

# saggitalT1画像の前処理

In [ ]:
import os
import pandas as pd

# データセットのパス
base_dir = '/content/train_images'

# リストを初期化
data = []

# 一番上の階層（study_id）のフォルダをたどる
for study_id in os.listdir(base_dir):
    study_path = os.path.join(base_dir, study_id)
    if os.path.isdir(study_path):
        # 二番目の階層（series_id）のフォルダをたどる
        for series_id in os.listdir(study_path):
            series_path = os.path.join(study_path, series_id)
            if os.path.isdir(series_path):
                # 三番目の階層（〇〇.dcm ファイル）をたどる
                for filename in os.listdir(series_path):
                    if filename.endswith('.dcm'):
                        # 〇〇.dcm の〇〇部分（instance_number）を抽出
                        instance_number = filename.split('.')[0]
                        # データを追加
                        data.append([study_id, series_id, instance_number])

# pandas DataFrameを作成
df = pd.DataFrame(data, columns=['study_id', 'series_id', 'instance_number'])



In [ ]:
df_meta_f = pd.read_csv('/content/train_series_descriptions.csv')
df_meta_f.tail()

df = df.astype('int64')

all_df = pd.merge(df, df_meta_f, on=['study_id', 'series_id'], how='inner')

s_all_df=all_df[all_df['series_description']=='Sagittal T1']

In [ ]:
import os
import numpy as np

# データフレームをコピーして操作
s_all_df_copy = s_all_df.copy()

# x_pos カラムを NaN で初期化
s_all_df_copy['x_pos'] = None
s_all_df_copy['y_pos'] = None
s_all_df_copy['z_pos'] = None
s_all_df_copy['pixel_sp_z'] = None
s_all_df_copy['pixel_sp_y'] = None
# 各行を処理
for idx, row in s_all_df_copy.iterrows():
    # DICOMファイルのパスを構築
    dicom_file_path = f"/content/train_images/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm"

    # DICOMファイルを読み込む
    dicom_data = pydicom.dcmread(dicom_file_path)

    # 'Image Position (Patient)' の x 座標を取得して x_pos カラムに格納
    s_all_df_copy.loc[idx, 'x_pos'] = dicom_data.ImagePositionPatient[0]
    s_all_df_copy.loc[idx, 'y_pos'] = dicom_data.ImagePositionPatient[1]
    s_all_df_copy.loc[idx, 'z_pos'] = dicom_data.ImagePositionPatient[2]
    s_all_df_copy.loc[idx, 'pixel_sp_z']=dicom_data.PixelSpacing[0]
    s_all_df_copy.loc[idx, 'pixel_sp_y']=dicom_data.PixelSpacing[1]



In [ ]:
import pandas as pd

# 並び替えとdescriptionカラムの更新を行う関数
def assign_description(group):
    # x_posを基準に小さい順に並び替え
    group = group.sort_values(by='x_pos').reset_index(drop=True)

    # 前半と後半を分割するインデックス
    midpoint = len(group) // 2

    # 前半を'Right Neural Foraminal Narrowing'
    group.loc[:midpoint-1, 'description'] = 'Right Neural Foraminal Narrowing'

    # 後半を'Left Neural Foraminal Narrowing'
    group.loc[midpoint:, 'description'] = 'Left Neural Foraminal Narrowing'

    return group

# study_id, series_idごとにグループ化し、assign_description関数を適用
s_all_df = s_all_df_copy.groupby(['study_id']).apply(assign_description).reset_index(drop=True)

<ipython-input-10-6a1f617f6ca8>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  s_all_df = s_all_df_copy.groupby(['study_id']).apply(assign_description).reset_index(drop=True)


In [ ]:
def sort_based_on_description(df):
    # "Right Neural Foraminal Narrowing" は x_pos が小さい順
    df_right = df[df['description'] == 'Right Neural Foraminal Narrowing'].sort_values(by='x_pos', ascending=True)

    # "Left Neural Foraminal Narrowing" は x_pos が大きい順
    df_left = df[df['description'] == 'Left Neural Foraminal Narrowing'].sort_values(by='x_pos', ascending=False)

    # 結合して返す
    return pd.concat([df_right, df_left])

# study_id, series_idごとにグループ化し、上記の関数で並び替え
sorted_s_all_df2 = s_all_df.groupby(['study_id', 'series_id'], group_keys=False).apply(sort_based_on_description)

# 結果を表示

<ipython-input-11-0af86ea5ca3a>:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_s_all_df2 = s_all_df.groupby(['study_id', 'series_id'], group_keys=False).apply(sort_based_on_description)


In [ ]:
import pandas as pd

# グループごとに均等な間隔で10個の行を選択する関数
def select_evenly_spaced(group, num=8):
    # グループのサイズが10個未満の場合はそのまま返す
    if len(group) <= num:
        return group
    # ステップを計算し、均等な間隔でデータを選ぶ
    step = len(group) / num
    indices = [int(i * step) for i in range(num)]
    return group.iloc[indices]

# study_id, instance_number, descriptionでグループ化し、各グループから均等に10個選択
new_df =sorted_s_all_df2.groupby(['study_id', 'description'], group_keys=False).apply(select_evenly_spaced)

# 結果の確認
new_df.head(20)

<ipython-input-12-2923b7af9ab3>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df =sorted_s_all_df2.groupby(['study_id', 'description'], group_keys=False).apply(select_evenly_spaced)


,study_id,series_id,instance_number,series_description,x_pos,y_pos,z_pos,pixel_sp_z,pixel_sp_y,description
14,4003253,1054713880,15,Sagittal T1,35.317043,-76.607254,-282.59006,0.78125,0.78125,Left Neural Foraminal Narrowing
13,4003253,1054713880,14,Sagittal T1,30.51716,-76.61295,-282.62344,0.78125,0.78125,Left Neural Foraminal Narrowing
12,4003253,1054713880,13,Sagittal T1,25.717281,-76.618645,-282.65683,0.78125,0.78125,Left Neural Foraminal Narrowing
11,4003253,1054713880,12,Sagittal T1,20.9174,-76.62434,-282.69022,0.78125,0.78125,Left Neural Foraminal Narrowing
10,4003253,1054713880,11,Sagittal T1,16.11752,-76.630035,-282.7236,0.78125,0.78125,Left Neural Foraminal Narrowing
9,4003253,1054713880,10,Sagittal T1,11.317637,-76.63573,-282.75696,0.78125,0.78125,Left Neural Foraminal Narrowing
8,4003253,1054713880,9,Sagittal T1,6.517758,-76.64142,-282.79034,0.78125,0.78125,Left Neural Foraminal Narrowing
7,4003253,1054713880,8,Sagittal T1,1.717877,-76.64711,-282.82373,0.78125,0.78125,Left Neural Foraminal Narrowing
0,4003253,1054713880,1,Sagittal T1,-31.881287,-76.68697,-283.0574,0.78125,0.78125,Right Neural Foraminal Narrowing
1,4003253,1054713880,2,Sagittal T1,-27.081406,-76.681274,-283.02402,0.78125,0.78125,Right Neural Foraminal Narrowing


In [ ]:
import pandas as pd
import numpy as np

# Function to ensure at least 8 rows in each group
def ensure_min_rows(group, min_rows=8):
    if len(group) < min_rows:
        # Randomly sample (with replacement) from the group to make up the missing rows
        additional_rows = group.sample(n=min_rows - len(group), replace=True, random_state=42)
        group = pd.concat([group, additional_rows], ignore_index=True)
    return group

# Apply the function to each group (grouped by 'study_id' and 'description')
new_df_padded = new_df.groupby(['study_id', 'description'], group_keys=False).apply(ensure_min_rows)

# Display the result
print(new_df_padded)

      study_id   series_id  instance_number series_description      x_pos  \
14     4003253  1054713880               15        Sagittal T1  35.317043   
13     4003253  1054713880               14        Sagittal T1   30.51716   
12     4003253  1054713880               13        Sagittal T1  25.717281   
11     4003253  1054713880               12        Sagittal T1    20.9174   
10     4003253  1054713880               11        Sagittal T1   16.11752   
..         ...         ...              ...                ...        ...   
3   4290709089  4237840455                4        Sagittal T1  -9.787976   
4   4290709089  4237840455                5        Sagittal T1  -5.188914   
5   4290709089  4237840455                6        Sagittal T1  -0.589853   
6   4290709089  4237840455                7        Sagittal T1   4.009209   
7   4290709089  4237840455                7        Sagittal T1   4.009209   

        y_pos       z_pos pixel_sp_z pixel_sp_y  \
14 -76.607254  -282.5900

<ipython-input-13-1ac8fd4931b5>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_df_padded = new_df.groupby(['study_id', 'description'], group_keys=False).apply(ensure_min_rows)


In [ ]:
import pandas as pd

# s_all_df3の5つのコピーを作成し、それぞれにlevelカラムを追加
dfs = [new_df_padded.copy().assign(level=i) for i in range(5)]

# それらのデータフレームを縦に連結
combined_df = pd.concat(dfs, ignore_index=True)

# 結果の確認
print(combined_df.head())
print(combined_df['level'].value_counts())

   study_id   series_id  instance_number series_description      x_pos  \
0   4003253  1054713880               15        Sagittal T1  35.317043   
1   4003253  1054713880               14        Sagittal T1   30.51716   
2   4003253  1054713880               13        Sagittal T1  25.717281   
3   4003253  1054713880               12        Sagittal T1    20.9174   
4   4003253  1054713880               11        Sagittal T1   16.11752   

       y_pos      z_pos pixel_sp_z pixel_sp_y  \
0 -76.607254 -282.59006    0.78125    0.78125   
1  -76.61295 -282.62344    0.78125    0.78125   
2 -76.618645 -282.65683    0.78125    0.78125   
3  -76.62434 -282.69022    0.78125    0.78125   
4 -76.630035  -282.7236    0.78125    0.78125   

                       description  level  
0  Left Neural Foraminal Narrowing      0  
1  Left Neural Foraminal Narrowing      0  
2  Left Neural Foraminal Narrowing      0  
3  Left Neural Foraminal Narrowing      0  
4  Left Neural Foraminal Narrowing      

In [ ]:
merged_df = pd.merge(combined_df,train, on='study_id', how='left')

In [ ]:
import pandas as pd
import numpy as np

# サンプルデータフレームの作成（ここでは仮のカラムを使用しています）
# 実際には combined_df には適切なカラムが含まれている必要があります
# combined_df = pd.DataFrame({
#     'description': [...],
#     'level': [...],
#     'left_neural_foraminal_narrowing_l1_l2': [...],
#     'left_neural_foraminal_narrowing_l2_l3': [...],
#     'left_neural_foraminal_narrowing_l3_l4': [...],
#     'left_neural_foraminal_narrowing_l4_l5': [...],
#     'left_neural_foraminal_narrowing_l5_s1': [...],
#     'right_neural_foraminal_narrowing_l1_l2': [...],
#     'right_neural_foraminal_narrowing_l2_l3': [...],
#     'right_neural_foraminal_narrowing_l3_l4': [...],
#     'right_neural_foraminal_narrowing_l4_l5': [...],
#     'right_neural_foraminal_narrowing_l5_s1': [...],
# })

# `label`カラムを作成
def get_label(row):
    if row['description'] == 'Left Neural Foraminal Narrowing':
        if row['level'] == 0:
            return row['left_neural_foraminal_narrowing_l1_l2']
        elif row['level'] == 1:
            return row['left_neural_foraminal_narrowing_l2_l3']
        elif row['level'] == 2:
            return row['left_neural_foraminal_narrowing_l3_l4']
        elif row['level'] == 3:
            return row['left_neural_foraminal_narrowing_l4_l5']
        elif row['level'] == 4:
            return row['left_neural_foraminal_narrowing_l5_s1']
    elif row['description'] == 'Right Neural Foraminal Narrowing':
        if row['level'] == 0:
            return row['right_neural_foraminal_narrowing_l1_l2']
        elif row['level'] == 1:
            return row['right_neural_foraminal_narrowing_l2_l3']
        elif row['level'] == 2:
            return row['right_neural_foraminal_narrowing_l3_l4']
        elif row['level'] == 3:
            return row['right_neural_foraminal_narrowing_l4_l5']
        elif row['level'] == 4:
            return row['right_neural_foraminal_narrowing_l5_s1']
    return np.nan  # 予期しない値の場合はNaNを返す

# 新しいカラム `label` を作成
merged_df['label'] = merged_df.apply(get_label, axis=1)

# 結果の確認
print(combined_df.head())

   study_id   series_id  instance_number series_description      x_pos  \
0   4003253  1054713880               15        Sagittal T1  35.317043   
1   4003253  1054713880               14        Sagittal T1   30.51716   
2   4003253  1054713880               13        Sagittal T1  25.717281   
3   4003253  1054713880               12        Sagittal T1    20.9174   
4   4003253  1054713880               11        Sagittal T1   16.11752   

       y_pos      z_pos pixel_sp_z pixel_sp_y  \
0 -76.607254 -282.59006    0.78125    0.78125   
1  -76.61295 -282.62344    0.78125    0.78125   
2 -76.618645 -282.65683    0.78125    0.78125   
3  -76.62434 -282.69022    0.78125    0.78125   
4 -76.630035  -282.7236    0.78125    0.78125   

                       description  level  
0  Left Neural Foraminal Narrowing      0  
1  Left Neural Foraminal Narrowing      0  
2  Left Neural Foraminal Narrowing      0  
3  Left Neural Foraminal Narrowing      0  
4  Left Neural Foraminal Narrowing      

In [ ]:
combined_df=merged_df

In [ ]:
import pandas as pd

# 例として final_df を作成（実際には適切なデータフレームを使用します）
# final_df = pd.DataFrame({
#     'description': [...],
#     'level': [...],
#     'label': [...],
#     'instance_number': [...],
#     'other_col1': [...],
#     'other_col2': [...],
#     ...
# })

# 1行にまとめるための処理

def group_rows(df, group_size):
    grouped = []
    num_groups = len(df) // group_size + int(len(df) % group_size != 0)

    for i in range(num_groups):
        start_idx = i * group_size
        end_idx = min((i + 1) * group_size, len(df))
        subset = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # グループ内の先頭行の値を取得
        first_row = subset.iloc[0]

        # 新しいカラムに instance_number を格納
        row_data = first_row.to_dict()  # 他の列の情報も含めて最初の行の情報を取得
        row_data.update({
            f'instance_number_{j}': subset.iloc[j]['instance_number'] for j in range(len(subset))
        })

        # フィルの中身をリストに格納
        grouped.append(row_data)

    return pd.DataFrame(grouped)

# グループサイズを設定（5つの行を1行にまとめる）
group_size = 8
grouped_df = group_rows(combined_df, group_size)

print(grouped_df.head())

   study_id   series_id  instance_number series_description      x_pos  \
0   4003253  1054713880               15        Sagittal T1  35.317043   
1   4003253  1054713880                1        Sagittal T1 -31.881287   
2   4646740  3486248476                1        Sagittal T1  35.946600   
3   4646740  3486248476               17        Sagittal T1 -16.853400   
4   7143189  3219733239               17        Sagittal T1  45.955800   

        y_pos      z_pos  pixel_sp_z  pixel_sp_y  \
0  -76.607254 -282.59006    0.781250    0.781250   
1  -76.686970 -283.05740    0.781250    0.781250   
2  -54.139700  155.92100    0.494792    0.494792   
3  -54.139700  155.92100    0.494792    0.494792   
4 -131.106000  164.24100    0.781250    0.781250   

                        description  ...  right_subarticular_stenosis_l5_s1  \
0   Left Neural Foraminal Narrowing  ...                        Normal/Mild   
1  Right Neural Foraminal Narrowing  ...                        Normal/Mild   
2   L

In [4]:
grouped_df=grouped_df[grouped_df['label'].notna()]

In [5]:
grouped_df.reset_index(drop=True, inplace=True)

In [6]:
# DataFrameを保存するパス
save_path = '/content/drive/MyDrive/RSNA_csv/grouped_df_rev.pkl'

# DataFrameをpkl形式で保存
grouped_df.to_pickle(save_path)

print(f"DataFrame has been saved to {save_path}")

DataFrame has been saved to /content/drive/MyDrive/RSNA_csv/grouped_df_rev.pkl


# saggitalT1画像の真ん中だけを取ってきたものの前処理

In [ ]:
import pandas as pd

# s_all_dfをstudy_idごとにグループ化し、instance_numberでソートし、真ん中の要素を取得
def get_middle_instance(df):
    middle_rows = []

    # study_idごとにグループ化
    grouped = df.groupby('study_id')

    for _, group in grouped:
        # instance_numberで昇順にソート
        sorted_group = group.sort_values(by='instance_number').reset_index(drop=True)

        # 真ん中のインデックスを計算
        midpoint = len(sorted_group) // 2

        # 真ん中の行を取得
        middle_row = sorted_group.iloc[midpoint]

        # 真ん中の行をリストに追加
        middle_rows.append(middle_row)

    # すべての真ん中の行を結合して新しいDataFrameを作成
    middle_df = pd.DataFrame(middle_rows)

    return middle_df

middle_s_all_df = get_middle_instance(s_all_df_copy)

# 結果を表示
print(middle_s_all_df)


      study_id   series_id  instance_number series_description      x_pos  \
7      4003253  1054713880                8        Sagittal T1   1.717877   
8      4646740  3486248476                9        Sagittal T1   9.546560   
8      7143189  3219733239                9        Sagittal T1   9.163940   
7      8785691  1570286759                8        Sagittal T1  18.931000   
9     10728036  2399638375               10        Sagittal T1 -30.544500   
..         ...         ...              ...                ...        ...   
10  4282019580  3029774733               11        Sagittal T1 -11.442256   
9   4283570761  2708429184               10        Sagittal T1  27.561708   
10  4284048608  1875151370               11        Sagittal T1  -7.587891   
7   4287160193   327893304                8        Sagittal T1 -11.264663   
7   4290709089  4237840455                8        Sagittal T1   8.608270   

         y_pos       z_pos  pixel_sp_z  pixel_sp_y  
7   -76.647110 -282.82

In [ ]:
middle_s_all_df =  middle_s_all_df.reset_index(drop=False)

In [ ]:
with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+".pkl", 'rb') as f:
      model1=pickle.load(f)

In [ ]:
import torch

# (OUT > 0.5)[0][1] がテンソルマスクだと仮定
for j in range(len(middle_s_all_df)):
  image=pydicom.dcmread("/content/train_images/"+str(middle_s_all_df.iloc[j]["study_id"])+"/"+str(middle_s_all_df.iloc[j]["series_id"])+"/"+str(middle_s_all_df.iloc[j]["instance_number"])+".dcm").pixel_array
  dicom=pydicom.dcmread("/content/train_images/"+str(middle_s_all_df.iloc[j]["study_id"])+"/"+str(middle_s_all_df.iloc[j]["series_id"])+"/"+str(middle_s_all_df.iloc[j]["instance_number"])+".dcm")
  H,W = image.shape
  middle_s_all_df.loc[j, 'H'] = H
  middle_s_all_df.loc[j, 'W'] = W
  middle_s_all_df.loc[j,'o0']=dicom.ImageOrientationPatient[0]
  middle_s_all_df.loc[j,'o1']=dicom.ImageOrientationPatient[1]
  middle_s_all_df.loc[j,'o2']=dicom.ImageOrientationPatient[2]
  middle_s_all_df.loc[j,'o3']=dicom.ImageOrientationPatient[3]
  middle_s_all_df.loc[j,'o4']=dicom.ImageOrientationPatient[4]
  middle_s_all_df.loc[j,'o5']=dicom.ImageOrientationPatient[5]
  middle_s_all_df.loc[j, 'H_cut']=0
  middle_s_all_df.loc[j, 'W_cut']=0
  # centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()
  # By plane resizing I've been distorting the proportions
  if H > W:
      d = W
      h = (H - d)//2
      image = image[h:h+d]
      middle_s_all_df.loc[j, 'H_cut'] = h
      # centers[:,1] -= h
      H = W
  elif H < W:
      d = H
      w = (W - d)//2
      image = image[:,w:w+d]
      middle_s_all_df.loc[j, 'W_cut'] = w
      # centers[:,0] -= w
      W = H
  image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
  image = torch.as_tensor(image/np.max(image)).unsqueeze(0).float()
  image=image.to(device)
  OUT = model1(image.unsqueeze(0)).cpu().detach()
  for i in range(5):
    mask = (OUT > 0.5)[0][i]  # 512x512のテンソル

    # Trueのピクセル座標を取得（y座標とx座標のペアとして取得）
    true_pixels = torch.nonzero(mask)

    # 平均座標を計算
    mean_yx = true_pixels.float().mean(dim=0)

    # 平均座標を表示
    mean_y, mean_x = mean_yx[0], mean_yx[1]
    print(f"Mean coordinates: (y, x) = ({0-mean_y.item()}, {mean_x.item()})")

    middle_s_all_df.loc[j, f'mean_y_{i}']=mean_y.item()
    middle_s_all_df.loc[j, f'mean_x_{i}']=mean_x.item()

    middle_s_all_df.loc[j, f'z_pos_{i}'] = middle_s_all_df.loc[j, 'pixel_sp_z'] * (0 - mean_y.item()) + middle_s_all_df.loc[j, 'z_pos']


ストリーミング出力は最後の 5000 行に切り捨てられました。
Mean coordinates: (y, x) = (-161.8973388671875, 270.7224426269531)
Mean coordinates: (y, x) = (-220.1776123046875, 261.8223876953125)
Mean coordinates: (y, x) = (-283.79254150390625, 256.1410827636719)
Mean coordinates: (y, x) = (-343.021728515625, 249.8478240966797)
Mean coordinates: (y, x) = (-398.6896667480469, 254.0)
Mean coordinates: (y, x) = (-165.6642608642578, 291.0)
Mean coordinates: (y, x) = (-218.76119995117188, 272.9664306640625)
Mean coordinates: (y, x) = (-277.10791015625, 254.76979064941406)
Mean coordinates: (y, x) = (-329.45220947265625, 241.0808868408203)
Mean coordinates: (y, x) = (-381.0815734863281, 248.80142211914062)
Mean coordinates: (y, x) = (-143.1095428466797, 284.992919921875)
Mean coordinates: (y, x) = (-197.44927978515625, 276.2391357421875)
Mean coordinates: (y, x) = (-251.14285278320312, 264.02630615234375)
Mean coordinates: (y, x) = (-301.8856201171875, 256.8856201171875)
Mean coordinates: (y, x) = (-356.9239196777344, 25

In [ ]:
import torch

# 各マスクの座標を元の画像サイズに戻すコード
for j in range(len(middle_s_all_df)):
    # 元の画像サイズ
    H = middle_s_all_df.loc[j, 'H']
    W = middle_s_all_df.loc[j, 'W']

    # 512にリサイズされた画像のサイズ
    resized_size = 512

    # 高さと幅のスケール（リサイズ前の座標に戻すためのスケール）
    if H<W:
      scale_y = H / resized_size
      scale_x = H / resized_size
    elif H>W:
      scale_y = W / resized_size
      scale_x = W / resized_size
    else:
      scale_y = W / resized_size
      scale_x = H / resized_size
    # 5つのマスクについて座標を戻す
    for i in range(5):
        # リサイズ後の座標を取得
        mean_y_resized = middle_s_all_df.loc[j, f'mean_y_{i}']
        mean_x_resized = middle_s_all_df.loc[j, f'mean_x_{i}']

        # リサイズ前の座標に戻す（スケールを掛ける）
        mean_bfr_y = mean_y_resized * scale_y
        mean_bfr_x = mean_x_resized * scale_x

        # 戻した座標を新しいカラムに格納
        middle_s_all_df.loc[j, f'mean_bfr_y_{i}'] = mean_bfr_y
        middle_s_all_df.loc[j, f'mean_bfr_x_{i}'] = mean_bfr_x


In [ ]:
import torch

# 各マスクの切り取る前の座標を計算して新しいカラムに格納
for j in range(len(middle_s_all_df)):
    # H_cut, W_cut はどれくらい切り取られたかを表す（NaNの場合は切り取りなし）
    H_cut = middle_s_all_df.loc[j, 'H_cut'] if not pd.isna(middle_s_all_df.loc[j, 'H_cut']) else 0
    W_cut = middle_s_all_df.loc[j, 'W_cut'] if not pd.isna(middle_s_all_df.loc[j, 'W_cut']) else 0

    for i in range(5):
        # リサイズ前の平均座標を取得
        mean_bfr_y = middle_s_all_df.loc[j, f'mean_bfr_y_{i}']
        mean_bfr_x = middle_s_all_df.loc[j, f'mean_bfr_x_{i}']

        # 切り取る前の座標を計算（H_cutとW_cutを戻す）
        mean_bfr_cut_y = mean_bfr_y + H_cut
        mean_bfr_cut_x = mean_bfr_x + W_cut

        # 切り取る前の座標を新しいカラムに格納
        middle_s_all_df.loc[j, f'mean_bfr_cut_y_{i}'] = mean_bfr_cut_y
        middle_s_all_df.loc[j, f'mean_bfr_cut_x_{i}'] = mean_bfr_cut_x


In [ ]:
import numpy as np

# 3次元座標を計算してカラムに格納するコード
for j in range(len(middle_s_all_df)):
    # DICOMのメタデータ
    x_pos = middle_s_all_df.loc[j, 'x_pos']
    y_pos = middle_s_all_df.loc[j, 'y_pos']
    z_pos = middle_s_all_df.loc[j, 'z_pos']

    o0, o1, o2 = middle_s_all_df.loc[j, ['o0', 'o1', 'o2']]  # 行方向ベクトル (ImageOrientationPatientの0〜2番目)
    o3, o4, o5 = middle_s_all_df.loc[j, ['o3', 'o4', 'o5']]  # 列方向ベクトル (ImageOrientationPatientの3〜5番目)

    pixel_sp_y = middle_s_all_df.loc[j, 'pixel_sp_y']  # 行方向のPixelSpacing
    pixel_sp_z = middle_s_all_df.loc[j, 'pixel_sp_z']  # 列方向のPixelSpacing

    # 行方向ベクトル
    row_direction = np.array([o0, o1, o2])
    # 列方向ベクトル
    col_direction = np.array([o3, o4, o5])

    # 各マスクに対して3次元座標を計算
    for i in range(5):
        # ピクセル座標を取得
        mean_y = middle_s_all_df.loc[j, f'mean_bfr_cut_y_{i}']  # 行方向（R）
        mean_x = middle_s_all_df.loc[j, f'mean_bfr_cut_x_{i}']  # 列方向（C）

        # 3次元座標を計算
        patient_pos = np.array([x_pos, y_pos, z_pos]) + \
                      (pixel_sp_y * mean_x) * row_direction + \
                      (pixel_sp_z * mean_y) * col_direction

        # 計算した3次元座標をカラムに格納
        middle_s_all_df.loc[j, f'xx_{i}'] = patient_pos[0]
        middle_s_all_df.loc[j, f'yy_{i}'] = patient_pos[1]
        middle_s_all_df.loc[j, f'zz_{i}'] = patient_pos[2]


In [ ]:
# DataFrameを保存するパス
save_path = '/content/drive/MyDrive/RSNA_csv/middle_s_all_df_rev.pkl'

# DataFrameをpkl形式で保存
middle_s_all_df.to_pickle(save_path)

print(f"DataFrame has been saved to {save_path}")

DataFrame has been saved to /content/drive/MyDrive/RSNA_csv/middle_s_all_df_rev.pkl


# 画像認識モデル学習部分

In [2]:
import pickle
with open("/content/drive/MyDrive/RSNA_csv/grouped_df_rev.pkl", 'rb') as f:
  grouped_df=pickle.load(f)

In [ ]:
unique_studies = grouped_df['study_id'].unique()
study_mapping = {study: (i % 5) + 1 for i, study in enumerate(unique_studies)}

# Map the numbers back to the original DataFrame
grouped_df['series_description2'] = grouped_df['study_id'].map(study_mapping)


In [ ]:
tdf2=grouped_df[grouped_df['series_description2'] != fold]
vdf2=grouped_df[grouped_df['series_description2'] == fold]

In [ ]:
import albumentations as A

AUG_PROB = 0.75
transforms_train = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        # A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=AUG_PROB),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=AUG_PROB),

    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
  #  A.CoarseDropout(max_holes=15, max_height=30, max_width=30, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),
])

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [ ]:
patch_size = 90

In [ ]:
import torch.nn.functional as F
class ViT_T1_Dataset(Dataset):
    def __init__(self, df, UNet,transform=None, VALID=False, P=patch_size, alpha=0):
        self.data = df
        self.UNet = UNet
        self.VALID = VALID
        self.P = P
        self.alpha = alpha
        self.transform = transform

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
      x = np.zeros((8, self.P, self.P), dtype=np.float32)
      non_zero_slice=[]

      for i in range(8):

        sample = '/content/train_images/'
        sample = sample+str(self.data.iloc[index]['study_id'])+'/'+str(self.data.iloc[index]['series_id'])+'/'+str(self.data.iloc[index][f'instance_number_{i}'])+'.dcm'

        image = pydicom.dcmread(sample).pixel_array
        H,W = image.shape
        # centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()
        # By plane resizing I've been distorting the proportions
        if H > W:
            d = W
            h = (H - d)//2
            image = image[h:h+d]
            # centers[:,1] -= h
            H = W
        elif H < W:
            d = H
            w = (W - d)//2
            image = image[:,w:w+d]
            # centers[:,0] -= w
            W = H
        image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
        image = torch.as_tensor(image/(np.max(image))).unsqueeze(0).unsqueeze(0).float().to(device)

        # OUT = 0
        # with torch.no_grad():
                # for rot in [0,1,2,3]:
                        # OUT += torch.rot90(self.UNet(torch.rot90(image,rot,dims=[-2, -1])),-rot,dims=[-2, -1])
        OUT=self.UNet(image)
        OUT = (OUT > TH)[0]
        c = (OUT.unsqueeze(1)*idx_map[0]).view(5,2,PATCH_SIZE*PATCH_SIZE).sum(-1)
        d = OUT.view(5,PATCH_SIZE*PATCH_SIZE).sum(-1)
        m = d > 0
        c[m] = (c[m]/(d[m]).unsqueeze(-1)).long()
        c[~m] = self.P # I have to find a better solution

        image_slices = []

        for xy in c:
          y_start = max(0, xy[1] - self.P // 2)
          y_end = min(512,xy[1] + self.P - self.P // 2)
          x_start = max(0, xy[0] - self.P // 2)
          x_end = min(512,xy[0] + self.P - self.P // 2)
          if (y_end - y_start == self.P) and (x_end - x_start == self.P):
            slice_img = image[0, 0, y_start:y_end, x_start:x_end]
            non_zero_slice.append(slice_img)
            image_slices.append(slice_img)
          else:
            zero_slice = torch.zeros((self.P, self.P), device=image.device)
            image_slices.append(zero_slice)
            # print(f"Skipped slice due to incorrect size: {(y_end - y_start, x_end - x_start)}")
            # print(f"Slice coordinates: {(xy[1], xy[0])}")

# スライスがあればスタックする
        if image_slices:
          try:
            image = torch.stack(image_slices)
          except RuntimeError as e:
            print(f"Error: {e}")
        else:
          print("No valid slices available for stacking.")

        # if not self.VALID: image = augment_image(image,self.alpha)
        x[i,...]=image[self.data.iloc[index]['level']].cpu().numpy()
      for i in range(8):
        if (x[i,...].sum() == 0) and (len(non_zero_slice)>0):
          x[i,...] = non_zero_slice[0].cpu().numpy()
        else:
          pass
        #nothing
      if self.transform is not None:
        x = self.transform(image=x)['image']
      x=torch.as_tensor(x).float()
      x = F.interpolate(x.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
      label = torch.as_tensor(labels[self.data.iloc[index]['label']])

      return [x.to(device),label.to(device)]


In [ ]:
def myLoss(preds,target):
    return nn.CrossEntropyLoss(weight=torch.as_tensor([1.,2.,4.]).to(device))(preds+1e-12,target)

In [ ]:
import timm
class ViT(nn.Module):
    def __init__(self, num_classes):
        super(ViT, self).__init__()
        # ここにViTのモデルアーキテクチャを実装
        self.vit = timm.create_model('eva02_base_patch14_224', pretrained=True, num_classes=num_classes,in_chans=8,features_only=False,global_pool='avg')
        # self.model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # self.vit.head.drop.p = 0.5
        #edgenext_base.in21k_ft_in1k
        # self.vit.features.conv0=nn.Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # self.new_model=nn.Sequential(*self.layer)
    def forward(self, x):
        # ここに順伝播の処理を実装
        return self.vit(x)

In [ ]:
from fastai.callback.core import Callback

class SaveModelCallback(Callback):
    def __init__(self, every_epoch=False, path='models', fname='model',with_opt=False):
        self.every_epoch = every_epoch
        self.path = path
        self.fname = fname
        self.with_opt = with_opt

    def after_epoch(self):
        # エポックごとにモデルを保存する
        if self.every_epoch:
            self.learn.save(f'{self.path}/{self.fname}_ep_{self.epoch}')

save_model_cb = SaveModelCallback(with_opt=True,every_epoch=True, path='/content/drive/MyDrive/RSNA_csv', fname=f"eva_p_90_ch_8_f_{fold}")

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from torch.optim import AdamW
if 1:
    seed_everything(SEED)
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+".pkl", 'rb') as f:
      UNet=pickle.load(f)
    tds = ViT_T1_Dataset(tdf2,UNet,transform=transforms_train)
    vds = ViT_T1_Dataset(vdf2,UNet,VALID=True)
    tdl = torch.utils.data.DataLoader(tds, batch_size=16, shuffle=True, drop_last=True)
    vdl = torch.utils.data.DataLoader(vds, batch_size=16, shuffle=False)

    dls = DataLoaders(tdl,vdl)

    n_iter = len(tds)//INF['BS']

    model = ViT(num_classes=3)
    model.to(device)
    learn = Learner(
        dls,
        model,
        lr=INF['LR'],
        loss_func=myLoss,
        cbs=[
            save_model_cb,
            GradientClip,
            ShowGraphCallback(),
            # alpha_cb
        ]
    )
    learn.fit_one_cycle(INF['EPOCHS'],wd=1e-2)
    with open('/content/drive/MyDrive/RSNA_csv/'+"VIT_"+str(fold)+".pkl", 'wb') as f:
      pickle.dump(model, f)

epoch,train_loss,valid_loss,time


KeyboardInterrupt: 